In [1]:
import time
import re
import pandas as pd
import numpy as np
import requests
import pickle
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pickle

입력어 받는 파이썬파일로.  
원하는리뷰수를 인풋?  
-이거계산하는거만들어보고  
url도 인풋으로

In [74]:
#스크롤 횟수

def Youtube_crawl(url, scroll_cnt):
    scroll_cnt = scroll_cnt

    url = url

    #자신의 크롬 버전에 맞는 드라이버 다운
    driver = webdriver.Chrome('./chromedriver') #크롬 드라이버 경로
    driver.get(url)

    time.sleep(2) #댓글창로딩

    body = driver.find_element_by_tag_name('body')

    while scroll_cnt:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        scroll_cnt -= 1

    reviews = driver.find_elements_by_xpath('//*[@id="main"]')
    likes = driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')
    print(scroll_cnt, len(reviews))


    if len(reviews) != len(likes):
        reviews = reviews[:-1] #마지막 review에 Youtube Premium 광고가 섞이는경우가있음

    df = pd.DataFrame(columns=['리뷰', '좋아요'])

    review_list = []
    like_list = []
    for review, like in zip(reviews, likes):
        review_list.append(review.text)
        like_list.append(like.text)

    df['리뷰'] = review_list
    df['좋아요'] = like_list

### page_down을 사용하면 버벅거리기에 느리다.
- page_down 대신에 scrollHeight 이용

In [ ]:
def Youtube_crawl_2(url):
    url = url

    #자신의 크롬 버전에 맞는 드라이버 다운
    driver = webdriver.Chrome('./chromedriver') #크롬 드라이버 경로
    driver.get(url)

    time.sleep(5) #댓글창로딩

    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    #댓글 최하단까지 이동
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
        time.sleep(5)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        if new_page_height == last_page_height:
            break

        last_page_height = new_page_height
    
    #자세히 보기를 클릭해 리뷰 잘림 현상 방지
    load_more = driver.find_elements_by_xpath('//*[@id="more"]/span')

    for load in load_more:
        if load.text == '자세히 보기':
            load.click()

    reviews = driver.find_elements_by_xpath('//*[@id="main"]')
    likes = driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')

    if len(reviews) != len(likes):
        reviews = reviews[:-1] #마지막 review에 Youtube Premium 광고가 섞이는경우가있음

    df = pd.DataFrame(columns=['리뷰', '좋아요'])

    review_list = []
    like_list = []
    for review, like in zip(reviews, likes):
        review_list.append(review.text)
        like_list.append(like.text)

    df['리뷰'] = review_list
    df['좋아요'] = like_list
    
    return df

### 답글이 있는 댓글을 못가져온다

In [284]:
def Youtube_crawl_3(url):
    url = url

    #자신의 크롬 버전에 맞는 드라이버 다운
    driver = webdriver.Chrome('./chromedriver') #크롬 드라이버 경로
    driver.get(url)

    time.sleep(5) #댓글창로딩

    #모든 댓글 불러오기
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
        time.sleep(5)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        if new_page_height == last_page_height:
            break

        last_page_height = new_page_height
    
    #대댓글 불러오기
    load_reply = driver.find_elements_by_selector('#contents #text')

    for load in load_reply:
        #답글을 남기는 '답글' 과 '답글 n개 보기' 중 'n개 보기' 필터링
        if '보기' in load.text: 
            driver.execute_script('arguments[0].click();',load)
            #load.click()의 경우 오류가 발생
        else:
            pass
    
    #대댓글 더 불러오기
    while True:
        try:
            check_point = driver.find_elements_by_xpath('//*[@id="continuation"]/yt-next-continuation/paper-button/yt-formatted-string')

            if len(check_point):
                #답글이 일정 수준을 넘기면 '답글 더보기'가 등장, 이를 클릭하기 위함
                load_more_reply = driver.find_elements_by_xpath('//*[@id="continuation"]/yt-next-continuation/paper-button/yt-formatted-string')

                for load in load_more_reply:
                    driver.execute_script('arguments[0].click();',load)
        except:
            break

    
    #답글에도 자세히보기를 사용해야할경우가있기때문에 자세히보기 클릭은 답글을 다 연 다음에
    load_detail = driver.find_elements_by_xpath('//*[@id="more"]/span')

    for load in load_detail:
        if load.text == '자세히 보기':
            driver.execute_script('arguments[0].click();',load)
        else:
            pass

    reviews = driver.find_elements_by_css_selector('#content-text')
    likes = driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')

    if len(reviews) != len(likes):
        reviews = reviews[:-1] #마지막 review에 Youtube Premium 광고가 섞이는경우가있음

    df = pd.DataFrame(columns=['리뷰', '좋아요'])

    review_list = []
    like_list = []
    for review, like in zip(reviews, likes):
        review_list.append(review.text)
        like_list.append(like.text)

    df['리뷰'] = review_list
    df['좋아요'] = like_list
    
    #좋아요 전처리
    df['좋아요'] = df['좋아요'].apply(lambda x: re.sub('천 | 만','000',x)) #천의 경우는 1천 / 만의 경우는 1.1만 의 경우로 표현됨
    
    #좋아요가 없는 칼럼은 1로 대체
    indexes = df[df['좋아요'] == ''].index
    df.loc[indexes, '좋아요'] = 1
    
    return df

대댓글 더 불러오는 과정에서 문제가 발생한다....

In [200]:
df = Youtube_crawl_3('https://www.youtube.com/watch?v=HhgcXuN0MlU&t=599s')

KeyboardInterrupt: 

In [272]:
url = 'https://www.youtube.com/watch?v=HhgcXuN0MlU&t=599s'

#자신의 크롬 버전에 맞는 드라이버 다운
driver = webdriver.Chrome('./chromedriver') #크롬 드라이버 경로
driver.get(url)

time.sleep(5) #댓글창로딩

#모든 댓글 불러오기
last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
    time.sleep(5)
    new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    if new_page_height == last_page_height:
        break

    last_page_height = new_page_height

In [209]:
reviews = driver.find_elements_by_xpath('//*[@id="main"]')
len(reviews)

1341

In [274]:
load_reply = driver.find_elements_by_css_selector('#contents #text')
# len(load_reply)

for load in load_reply:
    #답글을 남기는 '답글' 과 '답글 n개 보기' 두 가지 구분점을 '보기'로
    if '보기' in load.text: 
        #load.click()의 경우 오류가 발생
        driver.execute_script('arguments[0].click();',load) 
    else:
        pass

In [275]:
while True:
    try:
        check_point = driver.find_elements_by_xpath('//*[@id="continuation"]/yt-next-continuation/paper-button/yt-formatted-string')

        if len(check_point):
            #답글이 일정 수준을 넘기면 '답글 더보기'가 등장, 이를 클릭하기 위함
            load_more_reply = driver.find_elements_by_xpath('//*[@id="continuation"]/yt-next-continuation/paper-button/yt-formatted-string')

            for load in load_more_reply:
                driver.execute_script('arguments[0].click();',load)

    except:
        break

'자세히 보기'로 필터링

In [276]:
#답글에도 자세히보기를 사용해야할경우가있기때문에 자세히보기 클릭은 답글을 다 연 다음에
load_detail = driver.find_elements_by_xpath('//*[@id="more"]/span')

for load in load_detail:
    if load.text == '자세히 보기':
        driver.execute_script('arguments[0].click();',load)

In [213]:
reviews = driver.find_elements_by_xpath('//*[@id="main"]')
len(reviews)

2118

보다 많은 리뷰가 성공적으로 불러와졌다

In [277]:
reviews = driver.find_elements_by_css_selector('#content-text')
likes = driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')

if len(reviews) != len(likes):
    reviews = reviews[:-1] #마지막 review에 Youtube Premium 광고가 섞이는경우가있음

df = pd.DataFrame(columns=['리뷰', '좋아요'])

review_list = []
like_list = []
for review, like in zip(reviews, likes):
    review_list.append(review.text)
    like_list.append(like.text)

df['리뷰'] = review_list
df['좋아요'] = like_list

### 좋아요 수 전처리
- '1천', or ''

답글과 자세히보기문제

In [281]:
df['좋아요'] = df['좋아요'].apply(lambda x: re.sub('천 | 만','000',x)) #천의 경우는 1천 / 만의 경우는 1.1만 의 경우로 표현됨

#좋아요가 없는 칼럼은 1로 대체
indexes = df[df['좋아요'] == ''].index
df.loc[indexes, '좋아요'] = 1

In [283]:
df.to_pickle('유튜브 리뷰_2.pkl')

like에 가중치를 둔다면?
댓글 * like로 가중치를두어계산(이때 log2 씌워주고) #like가없다면 차이를두어야겠지?